In [1]:
require 'cunn';
require 'nn';
require 'paths'
local matio = require 'matio'
local tensor = matio.load('CIFAR2_reshaped.mat')
test_X = tensor.test_X

nRepeat = 4
labelLength = nRepeat * 2
numberTR0 = 5000
numberTR1 = 5000
numberTe0 = 500
numberTe1 = 500
label0 = torch.cat(torch.ones(nRepeat), torch.zeros(nRepeat),1)
label1 = torch.cat(torch.zeros(nRepeat), torch.ones(nRepeat),1)
label0 = label0:cuda()
label1 = label1:cuda()

train_Y = torch.cat(torch.repeatTensor(label0, numberTR0, 1), torch.repeatTensor(label1, numberTR1, 1), 1)
test_Y = torch.cat(torch.repeatTensor(label0, numberTe0, 1), torch.repeatTensor(label1, numberTe1, 1), 1)

tensors = {}
tensors.data = tensor.train_X
tensors.label = train_Y

--print(label0)
--print(train_Y[5001])
--print(torch.type(trainset))

In [2]:
-- ignore setmetatable for now, it is a feature beyond the scope of this tutorial. It sets the index operator.
setmetatable(tensors, 
    {__index = function(t, i) 
                    return {t.data[i], t.label[i]} 
                end}
);
tensors.data = tensors.data:double() -- convert the data from a ByteTensor to a DoubleTensor.

function tensors:size() 
    return self.data:size(1) 
end


--change to cuda after setmetatable. unknown reason
tensors.data = tensors.data:cuda()
tensors.label = tensors.label:cuda()


...e/chongmj/torch/install/share/lua/5.1/cutorch/Tensor.lua:14: $ Torch: not enough memory: you tried to allocate 0GB. Buy new RAM! at /home/chongmj/torch/pkg/torch/lib/TH/THGeneral.c:226
stack traceback:
	[C]: in function 'resize'
	...e/chongmj/torch/install/share/lua/5.1/cutorch/Tensor.lua:14: in function 'double'
	[string "-- ignore setmetatable for now, it is a featu..."]:7: in main chunk
	[C]: in function 'xpcall'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:209: in function </home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:173>
	/home/chongmj/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function

In [3]:
mean = {} -- store the mean, to normalize the test set in the future
stdv  = {} -- store the standard-deviation for the future
for i=1,3 do -- over each image channel
    mean[i] = tensors.data[{ {}, {i}, {}, {}  }]:mean() -- mean estimation
    print('Channel ' .. i .. ', Mean: ' .. mean[i])
    tensors.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction, :add(value) add value to all elements in place.
    
    stdv[i] = tensors.data[{ {}, {i}, {}, {}  }]:std() -- std estimation
    print('Channel ' .. i .. ', Standard Deviation: ' .. stdv[i])
    tensors.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling :div(value) divides all elements by value in place.
end

[string "mean = {} -- store the mean, to normalize the..."]:4: attempt to call method 'mean' (a nil value)
stack traceback:
	[string "mean = {} -- store the mean, to normalize the..."]:4: in main chunk
	[C]: in function 'xpcall'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:209: in function </home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:173>
	/home/chongmj/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 

In [4]:
net = nn.Sequential()
net:add(nn.SpatialZeroPadding(2,2,2,2))
net:add(nn.SpatialConvolution(3, 32, 5, 5)) -- 3 input image channels, 6 output channels, 5x5 convolution kernel
net:add(nn.Dropout(0.4))
net:add(nn.ReLU())                       -- non-linearity, think of this as sigmoid function 
net:add(nn.SpatialMaxPooling(2,2,2,2))     -- A max-pooling operation that looks at 2x2 windows and finds the max.

--split = nn.ConcatTable()

c = nn.Sequential()
c:add(nn.SpatialZeroPadding(2,2,2,2))
c:add(nn.SpatialConvolution(32, 32, 5, 5)) --repeat convolution
c:add(nn.Dropout(0.4))
c:add(nn.ReLU())                       -- non-linearity 
c:add(nn.SpatialMaxPooling(2,2,2,2))
c:add(nn.SpatialZeroPadding(2,2,2,2))
c:add(nn.SpatialConvolution(32, 64, 5, 5)) --repeat convolution
c:add(nn.Dropout(0.4))
c:add(nn.ReLU())                       -- non-linearity 
c:add(nn.SpatialMaxPooling(2,2,2,2))
c:add(nn.View(64*4*4))                    -- reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5
c:add(nn.Linear(64*4*4, 120))             -- fully connected layer (matrix multiplication between input and weights), neural network, full connection.
c:add(nn.ReLU())                       -- non-linearity 
c:add(nn.Linear(120, 84))
c:add(nn.ReLU())                       -- non-linearity 
c:add(nn.Dropout(0.5))
c:add(nn.Linear(84, labelLength/2))                   -- 10 is the number of outputs of the network (in this case, 10 digits)
c:add(nn.Sigmoid()) 

d = c

--net:add(nn.LogSoftMax())                     -- converts the output to a log-probability. Useful for classification problems

--split:add(c)
--split:add(c)

concat=nn.Concat(1)
concat:add(c)
concat:add(c)

net:add(concat)
net = net:cuda()

In [5]:
criterion = nn.MultiLabelSoftMarginCriterion()
criterion = criterion:cuda()
trainer = nn.StochasticGradient(net, criterion)
trainer.learningRate = 0.0005
trainer.maxIteration = 1500 -- just do 5 epochs of training.

In [6]:
trainer:train(tensors)

...mj/torch/install/share/lua/5.1/nn/StochasticGradient.lua:19: attempt to call method 'size' (a table value)
stack traceback:
	...mj/torch/install/share/lua/5.1/nn/StochasticGradient.lua:19: in function 'f'
	[string "local f = function() return trainer:train(ten..."]:1: in main chunk
	[C]: in function 'xpcall'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:209: in function </home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:173>
	/home/chongmj/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 

In [7]:
test_X = test_X:double()   -- convert from Byte tensor to Double tensor
for i=1,3 do -- over each image channel
    tensors.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction    
    tensors.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling
end

[string "test_X = test_X:double()   -- convert from By..."]:3: attempt to perform arithmetic on a nil value
stack traceback:
	[string "test_X = test_X:double()   -- convert from By..."]:3: in main chunk
	[C]: in function 'xpcall'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:209: in function </home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:173>
	/home/chongmj/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...e/chongmj/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/chongmj/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00406670: 

In [8]:
correct = 0
for i=1,numberTe0 do
    local prediction = net:forward(test_X[i])
    if (prediction - label0)*(prediction - label0)< (prediction - label1) * (prediction -label1) then
        correct = correct + 1
    end
end

for i=numberTe0+1,numberTe0 + numberTe1 do
    local prediction = net:forward(test_X[i])
    if (prediction - label1)*(prediction - label1) < (prediction - label0)*(prediction - label0) then
        correct = correct + 1
    end
end

In [9]:
print(correct, 100*correct/(numberTe0+numberTe1) .. ' % ')

0	0 % 	


* Build crazy graphs of networks: https://github.com/torch/nngraph
* Train on imagenet with multiple GPUs: https://github.com/soumith/imagenet-multiGPU.torch
* Train recurrent networks with LSTM on text: https://github.com/wojzaremba/lstm

* More demos and tutorials: https://github.com/torch/torch7/wiki/Cheatsheet

* Chat with developers of Torch: http://gitter.im/torch/torch7
* Ask for help: http://groups.google.com/forum/#!forum/torch7
